# 税収と経済成長の関係について

## econtip

2016年5月26日

[経済まめ知識](http://econ.mokuren.ne.jp/wiki)

税収弾性値が１９９０年代半ば以前と以後で変化していることをグラフで確認した後、
この前後の期間を通じて妥当であるような推測式を得るためラグ付きの回帰計算を行う。

* 使用したデータ： [Excel ファイル](./revenue-elasticity.xlsx)
* 使用したデータの出所：
  * [2009年度国民経済計算（2000年基準・93SNA）](http://www.esri.cao.go.jp/jp/sna/data/data_list/kakuhou/files/h21/h21_kaku_top.html)
  * [2014年度国民経済計算（2005年基準・93SNA）](http://www.esri.cao.go.jp/jp/sna/data/data_list/kakuhou/files/h26/h26_kaku_top.html)
  * [第4表　昭和57年度以降一般会計歳入主要科目別決算](http://www.mof.go.jp/budget/reference/statistics/data.htm)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from bokeh.plotting import figure, show, output_notebook, hplot
output_notebook()

Loading BokehJS ...

## データの準備

In [2]:
xls = pd.ExcelFile('revenue-elasticity.xlsx')
Pdf = xls.parse(xls.sheet_names[1], index_col=0)
Ddf = xls.parse(xls.sheet_names[0], index_col=0)

In [3]:
dfs = []
trev = Ddf['TREV']
ptrev = trev.shift(1)
gtrev = (trev - ptrev)/ptrev*100.0
for ngdp in ['NGDP00', 'NGDP05']:
    c = Ddf[ngdp]
    p = c.shift(1) # previous year
    g = (c - p)/p*100.0 #the growth rate of NGDP
    _df = Ddf[[ngdp, 'TREV']]
    _df['p' + ngdp] = p
    _df['g' + ngdp] = g
    _df['pTREV'] = ptrev
    _df['gTREV'] = gtrev
    dfs.append(_df.dropna(how='any'))
Ddf00 = dfs[0]; Ddf05 = dfs[1]

In [4]:
Ddf00.head()

,NGDP00,TREV,pNGDP00,gNGDP00,pTREV,gTREV
YEAR,,,,,,
1983,288.7727,32.358328,276.1628,4.566111,30.511125,6.054195
1984,308.2384,34.908363,288.7727,6.740838,32.358328,7.880614
1985,330.3968,38.198839,308.2384,7.188721,34.908363,9.426039
1986,342.2664,41.876805,330.3968,3.592529,38.198839,9.628476
1987,362.2967,46.797850,342.2664,5.852254,41.876805,11.751243


## 税収弾性値

税収の増加率に対する名目 GDP の増加率の比を税収弾性値と呼ぶ。
税収を $T$, 名目 GDP を $G$ とおくと、税収弾性値 $\epsilon$ は
$$
\epsilon = \dfrac{\Delta T \, / \, T}{\Delta G \, / \, G}
$$
と書ける。

対数関数の微分 $\Delta \log X = \Delta X / X$ に注意すると、$\epsilon$ は $\log T$ の変分と $\log G$ の変分の比であることが推察される。実際、
$$
\log T = \alpha + \beta \log G
$$
という回帰式が成り立つなら両辺の変分をとって $\epsilon = \beta$ である。

以下に、名目 GDP と税収の両対数グラフを描く。


In [5]:
lnngdp00 = np.log(Ddf00['NGDP00'])
lntrev00 = np.log(Ddf00['TREV'])
lnngdp05 = np.log(Ddf05['NGDP05'])
lntrev05 = np.log(Ddf05['TREV'])

In [6]:
s1 = figure(title=u'1983-1997年', 
           x_axis_label=u'log(名目GDP)', y_axis_label=u'log(税収)', 
           width=300, height=300)
s1.circle(lnngdp00.ix[1983:1997], lntrev00.ix[1983:1997], 
         color='red', alpha=0.5, legend=u'2000年基準')
s1.legend.location = "bottom_right"

s2 = figure(title=u'1998-2012年', 
           x_axis_label=u'log(名目GDP)', y_axis_label=u'log(税収)', 
           width=300, height=300)
s2.circle(lnngdp05.ix[1998:2012], lntrev05.ix[1998:2012], 
         color='blue', alpha=0.5, legend=u'2005年基準')

s2.legend.location = "bottom_right"

p=hplot(s1, s2)
show(p)

上記のグラフのそれぞれで回帰直線を引けばその傾きとして税収弾性値が求まるが、１９９０年代半ばを境に税収弾性値は変化しているようである。名目GDP２０００年基準と２００５年基準の両者のデータを１つのグラフに描くと次のようになる。

In [7]:
output_notebook()
p = figure(title=u'赤：2000年基準、青：2005年基準', 
           x_axis_label=u'log(名目GDP)', y_axis_label=u'log(税収)', 
           width=400, height=400)
p.circle(lnngdp00, lntrev00, 
         color='red', alpha=0.5, legend=u'1983-2009年')

p.circle(lnngdp05, lntrev05, 
         color='blue', alpha=0.5, legend=u'1995-2014年')

p.legend.location = "bottom_right"

show(p)

Loading BokehJS ...

下図に示されるように、１９９０年代前半の前後で税収の対 GDP 比が変化している。税収は GDP の水準のみでは決まらないと見てよいであろう。

In [8]:
p = figure(title=u'税収',
           x_axis_label = u'年度', y_axis_label = u'税収／GDP（%）', 
           width=400, height=300)
p.line(Ddf00.index, Ddf00['TREV']/Ddf00['NGDP00']*100.0, 
       legend=u'2000年基準', color='red', alpha=0.5)
p.line(Ddf05.index, Ddf05['TREV']/Ddf05['NGDP05']*100.0, 
       legend=u'2005年基準', color='blue', alpha=0.5)
p.legend.location = "top_right"
show(p)

## 税収回帰式

景気変動の税収への影響を取り入れるため、ラグを持つ回帰を行う。具体的には次のようなモデルを用いる：
$$
T_n = \alpha + \beta G_n + \gamma G_{n-1}.
$$
成長率を $g$ とすると $G_{n-1} = G_n/(1 + g) \simeq (1 - g)G_n$ であるから、これを上式に代入して
$$
T_n \simeq \alpha + (\beta + \gamma) G_n - g \gamma G_n
$$
を得る。右辺第３項 $-g \gamma G_n$ が成長率に対する依存を表す。このとき $\beta + \gamma > 0$, $\gamma < 0$ であることが予想される。

参考のため次のラグのない回帰計算も行い比較する：
$$
T_n = \alpha + \beta G_n.
$$




In [9]:
regsL0 = []; guessesL0 = []
for std, df in zip(['00', '05'], [Ddf00, Ddf05]):
    reg = sm.OLS(df['TREV'], sm.add_constant(df['NGDP'+std])).fit()
    regsL0.append(reg)
    p = reg.params
    gs = p[0] + p[1]*df['NGDP'+std]
    guessesL0.append(gs)

In [10]:
regsL1 = []; guessesL1 = []
for std, df in zip(['00', '05'], [Ddf00, Ddf05]):
    reg = sm.OLS(df['TREV'], 
                sm.add_constant(
                    np.column_stack(
                        [df['NGDP'+std], df['pNGDP'+std]])
                    )
                ).fit()
    regsL1.append(reg)
    p = reg.params
    gs = p[0] + p[1]*df['NGDP'+std] + p[2]*df['pNGDP'+std]
    guessesL1.append(gs)

In [11]:
p = figure(title=u'税収・回帰式',
          x_axis_label=u'年度', y_axis_label=u'税収',
          width=400, height=400)
p.line(Ddf.index, Ddf['TREV'], legend=u'税収',
       color='black', alpha=0.5)
p.line(Ddf00.index, guessesL0[0], legend=u'ラグなし', 
       color='red', alpha=0.5, line_dash=[4,4])
p.line(Ddf00.index, guessesL1[0], legend=u'ラグあり',
       color='blue', alpha=0.5, line_dash=[4,4])
p.legend.location='bottom_right'
show(p)

回帰計算のサマリーは以下の通り。
パラメータは $\alpha = 2.6267$, $\beta = 0.4629$, $\gamma = -0.3692$ である。

In [12]:
regsL1[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TREV   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     35.25
Date:                Tue, 31 May 2016   Prob (F-statistic):           7.20e-08
Time:                        19:01:21   Log-Likelihood:                -71.202
No. Observations:                  27   AIC:                             148.4
Df Residuals:                      24   BIC:                             152.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          2.6267      5.610      0.468      0.644        -8.952    14.205
x1             0.4629      0.066      7.034      0.000         0.327     0.599
x2            -0.3692      0.059     -6.250      0.000        -0.491    -0.247
==============================================================================
Omnibus:                        1.270   Durbin-Watson:                   1.014
Prob(Omnibus):                  0.530   Jarque-Bera (JB):                0.922
Skew:                          -0.110   Prob(JB):                        0.631
Kurtosis:                       2.122   Cond. No.                     5.27e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.27e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 参考
税収、名目GDP、名目GDP成長率のグラフ

In [13]:
output_notebook()
p = figure(title=u'一般会計税収', 
           x_axis_label = u'年度', y_axis_label=u'税収(兆円)',
           width=400, height=300)
p.line(Ddf.index, Ddf['TREV'], color='red', alpha=0.5, legend=u'税収')
p.legend.location = "bottom_right"
show(p)

Loading BokehJS ...

In [14]:
output_notebook()
p = figure(title=u'名目GDP',
           x_axis_label = u'年度', y_axis_label = u'GDP（兆円）', 
           width=400, height=300)
p.line(Ddf00.index, Ddf00['NGDP00'], 
       legend=u'2000年基準', color='red', alpha=0.5)
p.line(Ddf05.index, Ddf05['NGDP05'], 
       legend=u'2005年基準', color='blue', alpha=0.5)
p.legend.location = "bottom_right"
show(p)

Loading BokehJS ...

In [15]:
output_notebook()
p = figure(title=u'名目GDP成長率',
           x_axis_label = u'年度', y_axis_label = u'（%）', 
           width=400, height=300)
p.line(Ddf00.index, Ddf00['gNGDP00'], 
       legend=u'2000年基準', color='red', alpha=0.5)
p.line(Ddf05.index, Ddf05['gNGDP05'], 
       legend=u'2005年基準', color='blue', alpha=0.5)
p.legend.location = "top_right"
show(p)

Loading BokehJS ...